In [1]:
import sys
sys.path.append('./src')

import os

import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets.hyperion_dataset import HyperionDataset
from trainers.mp_trainer import MPTrainer
from utils.utils import seed_everything

batch_size = 2

seed_everything(4321)

#cluster paths
#df = pd.read_csv('./RepML/data/Splitted_full/Hyperion_train.csv', na_filter=False)
#test_df = pd.read_csv('./RepML/data/Splitted_full/Hyperion_test.csv', na_filter=False)

#local paths
df = pd.read_csv('./data/Splitted_full/Hyperion_train.csv', na_filter=False)
test_df = pd.read_csv('./data/Splitted_full/Hyperion_test.csv', na_filter=False)

model_name = "dbmdz/bert-base-italian-xxl-uncased"
test_dataset = HyperionDataset(test_df.head(100), model_name)

learning_rate = 1e-5
batch_size = 2
n_epochs = 1

trainer = MPTrainer(batch_size, learning_rate, n_epochs, torch.nn.NLLLoss())

model = AutoModelForSequenceClassification.from_pretrained('MiBo/RepML')



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/mibo8/Rep/e/REP-200
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [2]:
pred, gt = trainer.test(model,test_dataset)


Running Test...
  Test Loss: 2.54
  Test took: 0:00:17


In [14]:
df = test_df.head(100)

In [15]:
error_dict = {
    'Stralcio' : df['Stralcio'].tolist(),
    'Repertorio_corretto' : decode_labels(gt),
    'Repertorio_predetto' : decode_labels(pred)
}

error_df = pd.DataFrame.from_dict(error_dict)

In [16]:
error_df

,Stralcio,Repertorio_corretto,Repertorio_predetto
0,aia ha anche affrontato il tema dei centri est...,0,tensor(7)
1,e il suo vaccino che mette a disposizione di a...,0,tensor(17)
2,Se anche le altre regioni facessero lo stesso...,0,tensor(15)
3,Se la sperimentazione riesce questo tipo di ta...,0,tensor(15)
4,"#Veneto, #Zaia: “#Lockdown sarebbe una tragedia,",0,tensor(17)
...,...,...,...
95,"Alla fine, non è stata la quarantena a limitar...",1,tensor(1)
96,e non possiamo intubare tanti pazienti perché ...,1,tensor(11)
97,Non si registrano grossi problemi se non quell...,1,tensor(7)
98,«Non sono in quarantena perché ho incontrato ...,1,tensor(20)
